In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

model = os.getenv("AZUREAI_CHAT_MODEL")
azure_endpoint = os.getenv("AZUREAI_CHAT_BASE_ENDPOINT")
azure_deployment = os.getenv("AZUREAI_CHAT_DEPLOYMENT")
azure_api_version = os.getenv("AZUREAI_CHAT_API_VERSION")
api_key = os.getenv("AZUREAI_CHAT_KEY")


In [2]:
from guidance import models, gen
from guidance import user, assistant
import tiktoken

def select(azureai_model, choices):
    tokenizer = tiktoken.encoding_for_model(model)
    encoded_choices = tokenizer.encode_batch(choices)
    encoded_choices_tokens = [token for choice in encoded_choices for token in choice]
    logit_bias = {str(token): +98 for token in encoded_choices_tokens}
    logit_bias["100257"] = 100  # "<|endoftext|>"
    azureai_model.set_logit_bias(logit_bias)

    max_tokens = max(len(choice) for choice in encoded_choices)
    return max_tokens



In [3]:
azureai_model = models.AzureOpenAI(
    model=model,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    version=azure_api_version,
    api_key=api_key,
    compute_log_probs=True,
)

choices = ["apples", "potatos", "I don't know", "hay", "carrots", "lettuce", "grass"]
max_tokens = select(azureai_model, choices)

with user():
    lm = azureai_model + "Which food does a rabbit prefer?"

with assistant():
    lm += gen("response", max_tokens=max_tokens)


In [4]:
azureai_model = models.AzureOpenAI(
    model=model,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    version=azure_api_version,
    api_key=api_key,
    compute_log_probs=True,
)

choices = ["beauty", "complicated", "love", "Love", "growth", "purpose", "42", "connection", "I don't know"]
max_tokens = select(azureai_model, choices)

with user():
    lm = azureai_model + "What is the meaning of life?"

with assistant():
    lm += gen("response", max_tokens=max_tokens)